In [1]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
###################################################################################################################

import xml.etree.ElementTree as etree
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"
###################################################################################################################
def format_variables(model):
    variables       = model.find(source_xmlns + 'variables')
    stocks          = variables.findall(source_xmlns + 'stock')
    auxs_and_ctes   = variables.findall(source_xmlns + 'aux')
    flows           = variables.findall(source_xmlns + 'flow')
    dependencies    = variables.find(source_xmlns_isee + 'dependencies')
    dependencies    = dependencies.findall(source_xmlns + 'var') if dependencies is not None else [] 
    dependencies_ = {}
    def get_params(eqn):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        return res

    for dep in dependencies:
        name = dep.get('name')
        inputs = dep.findall(source_xmlns + 'in')
        inputs_ = []
        for in_dep in inputs:
            inputs_.append(in_dep.text)
        dependencies_.update({name : inputs_})

    auxs_ = {}
    vars_with_dependencies = dependencies_.keys()
    for var in auxs_and_ctes:
        name = var.get('name')
        eqn  = var.find(source_xmlns + 'eqn').text
        if name in vars_with_dependencies:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})
        else:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})

    stocks_ = {}
    for stock in stocks:
        name = stock.get('name')
        eqn  = stock.find(source_xmlns + 'eqn').text
        inflows  = list(map(lambda x : x.text, stock.findall(source_xmlns + 'inflow')))
        outflows = list(map(lambda x : x.text, stock.findall(source_xmlns + 'outflow')))
        non_negative = True if stock.find(source_xmlns + 'non_negative') is not None else False
        stocks_.update({name : {
            'eqn' : eqn, 
            'inflows' : inflows, 
            'outflows' : outflows,
            'non_negative' : non_negative
        }})

    flows_ = {}
    for flow in flows:
        name = flow.get('name')
        eqn  = flow.find(source_xmlns + 'eqn').text
        flows_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})

    return {
        'dependencies' : dependencies_,
        'stocks' : stocks_,
        'flows' : flows_,
        'auxs'  : auxs_
    }

def format_variables_top_model(top_model):
    variables = top_model.find(source_xmlns + 'variables')
    modules = variables.findall(source_xmlns + 'module')
    auxs = variables.findall(source_xmlns + 'aux')
    modules_ = {}
    for module in modules:
        name = module.get('name')
        connections = module.findall(source_xmlns + 'connect')
        connections_ = []
        for connection in connections:
            connections_.append({'from' : connection.get('from'), 'to' : connection.get('to')})
        modules_.update({name : connections_})
    auxs_ = {}

    for aux in auxs:
        name = aux.get('name')
        eqn  = aux.find(source_xmlns + 'eqn').text
        auxs_.update({name : eqn})
    return {
        'modules' : modules_,
        'auxs' : auxs_
    }
###################################################################################################################
def generateDEVSML(formatted_variables, dst_xml):
    
    TEMPLATE_DEVSML = 'template-coupled-devsml.xml'
    
    auxs = formatted_variables['auxs']
    flows = formatted_variables['flows']
    stocks= formatted_variables['stocks']
    dependencies = formatted_variables['dependencies']

    TEMPLATE_COUPLED_DEVSML = 'template-coupled-devsml.xml'

    all_variables = stocks.keys() + flows.keys() + auxs.keys()
    internal_connections = []
    atomics = []

    #####################
    # Atomicos
    for stockName, stockAttr in stocks.iteritems():
        atomics.append({
            'name' : stockName + 'Tot',
            'model' : 'Ftot',
            'in_ports' : [{'name' : elem + 'Plus' , 'type' : 'in_plus'} for elem in stockAttr['inflows']] +
                         [{'name' : elem + 'Minus', 'type' : 'in_minus'} for elem in stockAttr['outflows']],
            'out_ports' : [{'name' :'out', 'type' : 'out'}]
        })
        atomics.append({
            'name' : stockName,
            'model' : 'QSS1',
            'in_ports' : [{'name' : 'in', 'type' : 'in'}],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'x0', 'function' : stockAttr['eqn']},
                {'name' : 'dQRel', 'function' : 1e-4},
                {'name' : 'dQMin', 'function' : 1e-4}
            ]
        })
    for auxName, auxAttr in auxs.iteritems():
        eqn_params = filter(lambda x : x in all_variables, auxAttr['eqn_params'])

        atomics.append({
            'name' : auxName,
            'model' : 'Cte' if eqn_params == [] else 'Aux',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in eqn_params],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : auxAttr['eqn']}
            ]
        })

    for flowName, flowAttr in flows.iteritems():
        atomics.append({
            'name' : flowName + 'Plus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })
        atomics.append({
            'name' : flowName + 'Minus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })

    #####################
    # Conexiones internas

    # Ftot => Integrador
    for stockName, stockAttr in stocks.iteritems():
        internal_connections.append({
            'component_from' : stockName + 'Tot',
            'component_to' : stockName,
            'port_from' : 'out',
            'port_to' : 'in'
        })
    # Fpm => Ftot
    for flowName, flowAttr in flows.iteritems():
        for stockName, stockAttr in stocks.iteritems():
            if flowName in stockAttr['inflows']:
                internal_connections.append({
                    'component_from' : flowName + 'Plus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Plus'
                })
            if flowName in stockAttr['outflows']:
                internal_connections.append({
                    'component_from' : flowName + 'Minus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Minus'
                })
    # Stock => {Flow, Aux}
    for stockName, stockAttr in stocks.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if stockName in flowAttr['eqn_params']:# and flowName in stockAttr['inflows']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
            if stockName in flowAttr['eqn_params']:# and flowName in stockAttr['outflows']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
        # Aux
        for auxName, auxAttr in auxs.iteritems():
            if stockName in auxAttr['eqn_params']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : auxName,
                    'port_from' : 'out',
                    'port_to' : stockName
                })
    # Aux => {Aux, Flow}
    for auxName, auxAttr in auxs.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if auxName in flowAttr['eqn_params']:
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : 'out',
                    'port_to' : auxName
                })
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : 'out',
                    'port_to' : auxName
                })
        # Aux
        for auxName2, auxAttr2 in auxs.iteritems():
            if auxName in auxAttr2['eqn_params'] and auxName != auxName2:
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : auxName2,
                    'port_from' : 'out',
                    'port_to' : auxName
                })

    #####################
    # Input Ports : los Aux's que no tienen valor, o que tienen un valor numerico
    in_ports = []
    for auxName, auxAttr in auxs.iteritems():
        # NOTA : los que estan vacios ASUMO que es porque viene de un modelo acoplado de afuera
        emptyEqn = (auxAttr['eqn'] == '')
        if emptyEqn or all([not elem in all_variables for elem in auxAttr['eqn_params']]):
            in_ports.append(auxName)

    # Output Ports : en ppio., todos los stocks
    out_ports = []
    for stockName, stockAttr in stocks.iteritems():
        out_ports.append(stockName)

    context = {
        'coupled_model_name' : 'top',
        'in_ports' : in_ports,
        'out_ports' : out_ports,
        'atomics' : atomics,
        'internal_connections' : internal_connections,
        'external_input_connections' : [],
        'external_output_connections' : []
    }
    z = render_template(TEMPLATE_COUPLED_DEVSML, context)
    with open(dst_xml, 'w') as f:
        f.write(render_template(TEMPLATE_DEVSML,context))
        
    return True
#################################################################################################################

In [2]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import xml.etree.ElementTree as etree
import os

PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
###################################################################################################################
def getContext(root):
    def get_params(eqn, all_variables):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        res = filter(lambda x : x in all_variables, res)
        return res
    #
    inputs = root.find('inputs')
    outputs = root.find('outputs')
    internal_connections = root.find('internal_connections')
    external_input_connections = root.find('external_input_connections')
    external_output_connections = root.find('external_output_connections')
    components = root.find('components')
    atomics = components.findall('atomicRef')
    all_variables = []
    for atomic in atomics:
        all_variables.append(atomic.get('name'))
    
    coupled = components.findall('coupled')
    
    # TODO : coupled_
    coupled_ = []

    #
    inputs_ = []
    outputs_ = []
    internal_connections_ = []
    external_input_connections_ = []
    external_output_connections_ = []
    atomics_ = []
    # Atomics
    for atomic in atomics:
        name  = atomic.get('name')
        model = atomic.get('model')
        ports = atomic.findall('port')
        parameters = atomic.findall('parameter')
        
        # TODO : MEJORAR ESTO
        class_ = name
        if model == 'QSS1':
            class_ = 'QSS1'
        if model == 'Cte':
            class_ = 'Cte'
        #
        atomics_.append({
            'name' : name,
            'class' : class_,
            'model' : model,
            'in_ports' : [{'name' : port.get('name'), 'type' : port.get('type')} 
                          for port in ports 
                              if port.get('type') == 'in' or
                                 port.get('type') =='in_plus' or
                                 port.get('type') == 'in_minus'],
            'out_ports' :  [{'name' : port.get('name'), 'type' : port.get('type')} 
                          for port in ports if port.get('type') == 'out'],
            'parameters' : [{'name' : param.get('name'), 
                             'function' : param.get('function'),
                             'function_params' : get_params(param.get('function'), all_variables)} 
                            for param in parameters]
        })
    
    # Internal Connections
    for connection in internal_connections:
        internal_connections_.append({
            'component_from' : connection.get('component_from'),
            'component_to' : connection.get('component_to'),
            'port_from' : connection.get('port_from'),
            'port_to' : connection.get('port_to')
        })
    # Input ports
    for in_port in inputs:
        inputs_.append(in_port.get('name'))
    # Output ports
    for out_port in outputs:
        outputs_.append(out_port.get('name'))
    context = {
        'inputs' : inputs_,
        'outputs' : outputs_,
        'atomics' : atomics_,
        'internal_connections' : internal_connections_,
        'external_input_connections' : external_input_connections_,
        'external_output_connections' : external_output_connections_
    }
    return context
###################################################################################################################
def generate_h_cpp_ev(context, folder):
    #
    def save(template, ext, context):
        srcFolder = folder + 'src/' + context['name'] + ext
        with open(srcFolder, 'wb') as f:
            f.write(render_template(template + ext, context))
    #
    ctes = filter(lambda x : x['model'] == 'Cte', context['atomics'])
    auxs = filter(lambda x : x['model'] == 'Aux', context['atomics'])
    fpms = filter(lambda x : x['model'] == 'Fpm', context['atomics'])
    fts  = filter(lambda x : x['model'] == 'Ftot', context['atomics'])

    for aux in auxs:
        parameters = aux['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        aux['parameters'] = parameters_
    
    for fpm in fpms:
        parameters = fpm['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        fpm['parameters'] = parameters_
    
    for ft in fts:
        in_ports_plus, in_ports_minus = [], []
        parameters = ft['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        in_ports_plus = filter(lambda x : x['type'] == 'in_plus', ft['in_ports'])
        in_ports_minus = filter(lambda x : x['type'] == 'in_minus', ft['in_ports'])
        ft['parameters'] = parameters_
        ft['in_ports_plus'] = in_ports_plus
        ft['in_ports_minus'] = in_ports_minus
    # Fpms
    for fpm in fpms:
        template = 'template-Aux-Fpm'
        for ext in ['.h', '.cpp']:
            save(template, ext, fpm)
    
    # Auxs
    for aux in auxs:
        template = 'template-Aux-Fpm'
        for ext in ['.h', '.cpp']:
            save(template, ext, aux)
    # Ftot's
    for ft in fts:
        template = 'template-Ftot'
        for ext in ['.h', '.cpp']:
            save(template, ext, ft)
            
    # Reg-file
    template = 'template-reg.cpp'
    with open(folder + 'src/' + 'reg.cpp', 'wb') as f:
        context = {'atomics' : [atomic['name'] for atomic in auxs + fpms + fts]}
        f.write(render_template(template, context))
        
    # Ev-file
    ev = []
    for cte in ctes:
        assert len(cte['parameters']) == 1
        param = cte['parameters'][0]
        if param['function_params'] == []:
            ev.append({'name' : cte['name'], 'value' : param['function']})
    template = 'template-ev.ev'
    with open(folder + 'events.ev', 'wb') as f:
        f.write(render_template(template, {'variables' : ev}))
    
    return True

def generate_ma(context, dst_file):
    template = 'template-ma.ma'
    with open(dst_file, 'wb') as f:
        f.write(render_template(template, context))
    return True
###################################################################################################################

# Traducciones

## Goodwin - Version 1

In [93]:
DIR_XMILE  = 'goodwin/goodwin-1.xmile'
DIR_DEVSML = 'goodwin/goodwin-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp_ev(context, 'goodwin/')
generate_ma(context, 'goodwin/goodwin-traduccion.ma')

Start:  1  Stop:  100


True

## Goodwin - Version Minsky

In [94]:
DIR_XMILE  = 'goodwin-minsky/goodwin-minsky.xmile'
DIR_DEVSML = 'goodwin-minsky/goodwin-minsky-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp_ev(context, 'goodwin-minsky/')
generate_ma(context, 'goodwin-minsky/goodwin-minsky-traduccion.ma')

Start:  1  Stop:  55


True

# Lotka - Volterra

In [92]:
DIR_XMILE  = 'lotka-volterra/lotka-volterra.xmile'
DIR_DEVSML = 'lotka-volterra/lotka-volterra-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp_ev(context, 'lotka-volterra/')
generate_ma(context, 'lotka-volterra/lotka-volterra-traduccion.ma')

Start:  1  Stop:  400


True

# SIR

In [91]:
DIR_XMILE  = 'sir/sir.xmile'
DIR_DEVSML = 'sir/sir-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp_ev(context, 'sir/')
generate_ma(context, 'sir/sir-traduccion.ma')

Start:  1  Stop:  15


True

# Teacup

In [86]:
DIR_XMILE  = 'teacup/teacup.xmile'
DIR_DEVSML = 'teacup/teacup-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp_ev(context, 'teacup/')
generate_ma(context, 'teacup/teacup-traduccion.ma')

Start:  1  Stop:  13


True

# Dueling Loops

In [4]:
DIR_XMILE  = 'dueling-loops/dueling-loops.xmile'
DIR_DEVSML = 'dueling-loops/dueling-loops-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML)

print 'Start: ', start, ' Stop: ', stop
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)

############# PARA DUELING-LOOPS SOLO ##############
#generate_h_cpp_ev(context, 'dueling-loops/')
#generate_ma(context, 'dueling-loops/dueling-loops-traduccion.ma')

############# PARA MODELO DE OPINION ###############
generate_h_cpp_ev(context, 'opinion/')
generate_ma(context, 'opinion/opinion-dueling-loops-traduccion.ma')

Start:  1  Stop:  13


True